In [257]:
from pyspark.sql import SparkSession, Row, Column
from pyspark.sql.functions import (regexp_extract, count, countDistinct, sumDistinct, 
                                   lag, when, udf, sum, mean, avg, desc,min, max)
import datetime
import calendar
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DataType, TimestampType
from pyspark import SparkContext, SparkConf


from pyspark.sql.window import Window


<b>creating session and reading log file</b>

In [2]:
sp = SparkSession.builder.appName('weblog').getOrCreate()
sc = SparkContext.getOrCreate()
rddf = sc.textFile('2015_07_22_mktplace_shop_web_log_sample.log')



<b>Function to extract required attributes such as timestamp hour minute second host and url.
This function below is assuming the constant pattern</b>

In [278]:
def CreatRows(line):
    words = line.split(" ")

    # Convert time to TimeStamp format
    datetime_time = datetime.datetime.strptime(words[0], '%Y-%m-%dT%H:%M:%S.%fZ')
    
    # Convert time to timestamp format
    timestamp = calendar.timegm(datetime_time.utctimetuple())                 
    
    # Extract IP and port of client
    host, client_port = words[2].split(":")
    
    # Create Row object
    row = Row(
        #originalstamp = words[0], #complete timestamp
        timestamp = timestamp,           # Timestamp
              hour = datetime_time.hour,       # Hour from the timestamp
              minute = datetime_time.minute,   # Minute from the timestamp
              second = datetime_time.second,
              host = host, 
              url = words[12]) 
    
    return row


<b>Function based on regex extract. Hence not assuming the pattern. This works on individual rows but not on full dataframe. Still to debug</b>

In [280]:
def Reg_extract_att(line):
    try:
        host = re.search(host_pat, line).group(1)
    except:
        host = None
    #method = re.search(req_pat, line).group(1)
    try:
        request = re.search(req_pat, line).group(2)
    except:
        request = None
    #protocol = re.search(req_pat, line).group(3)
    try:
        stamp = re.search(time_pat, line).group(1)
    except:
        stamp = None
    #status = re.search(status_pat, line).group(1)
    
    # Convert time to TimeStamp format
    datetime_time = datetime.datetime.strptime(stamp, '%Y-%m-%dT%H:%M:%S.%fZ')
    
    # Convert time to timestamp format
    timestamp = calendar.timegm(datetime_time.utctimetuple())
    # Create Row object
    row = Row(timestamp = timestamp,           # Timestamp
              hour = datetime_time.hour,       # Hour from the timestamp
              minute = datetime_time.minute,   # Minute from the timestamp
              second = datetime_time.second,
              host = host, 
              url = request,
            # status = status
             ) 
    
    return row

In [225]:
type(calendar.timegm(datetime.datetime.strptime('2015-07-22T09:00:27.894580Z','%Y-%m-%dT%H:%M:%S.%fZ').utctimetuple()))

int

In [281]:
schema = StructType([
        #StructField("originalstamp", LongType(), True),
        StructField("timestamp", LongType(), True),
        StructField("hour", IntegerType(), True),
        StructField("minute", IntegerType(), True),
        StructField("second", IntegerType(), True),
        #StructField("status", StringType(), True),
        StructField("host", StringType(), True),
        StructField("url", StringType(), True)])

In [282]:
row_log_data = rddf.map(lambda row: CreatRows(row))
log_df = sp.createDataFrame(row_log_data, schema)

In [10]:
#log_df.na.drop(how = 'any')

log_df.where('timestamp is null').show()

log_df = log_df.na.drop(how = 'any')

+---------+----+------+------+----+---+
|timestamp|hour|minute|second|host|url|
+---------+----+------+------+----+---+
+---------+----+------+------+----+---+



In [11]:
log_df.createOrReplaceTempView('people')

In [18]:
temp_df = sp.sql('select *, lag(timestamp, 1, 0) over (partition by host order by timestamp) as prev_timestamp \
from people')

temp_df.createOrReplaceTempView('First_table')

#temp_df.filter(temp_df['prev_timestamp'] == 0).show()

#temp_df.show()

+----------+----+------+------+------------+--------------------+--------------+
| timestamp|hour|minute|second|        host|                 url|prev_timestamp|
+----------+----+------+------+------------+--------------------+--------------+
|1437581676|  16|    14|    36|1.186.143.37|https://paytm.com...|             0|
|1437533011|   2|    43|    31|1.187.164.29|https://paytm.com...|             0|
|1437583379|  16|    42|    59|  1.22.41.76|https://paytm.com...|             0|
|1437586901|  17|    41|    41| 1.23.208.26|https://paytm.com...|             0|
|1437542064|   5|    14|    24| 1.23.36.184|https://paytm.com...|             0|
|1437581498|  16|    11|    38|   1.38.19.8|https://paytm.com...|             0|
|1437555638|   9|     0|    38|  1.38.20.34|https://paytm.com...|             0|
|1437588321|  18|     5|    21|  1.39.13.13|https://paytm.com...|             0|
|1437562828|  11|     0|    28| 1.39.32.249|https://paytm.com...|             0|
|1437583315|  16|    41|    

In [19]:
Reconstruction_df = sp.sql('select *, ROW_NUMBER() over (partition by host order by timestamp) as Sessionum from\
(select * from First_table where (timestamp -prev_timestamp) >=600 or prev_timestamp = 0) sometable')


Reconstruction_df.createOrReplaceTempView('Rec_table')

In [20]:
Reconstruction_df.show()

+----------+----+------+------+------------+--------------------+--------------+---------+
| timestamp|hour|minute|second|        host|                 url|prev_timestamp|Sessionum|
+----------+----+------+------+------------+--------------------+--------------+---------+
|1437581676|  16|    14|    36|1.186.143.37|https://paytm.com...|             0|        1|
|1437533011|   2|    43|    31|1.187.164.29|https://paytm.com...|             0|        1|
|1437583379|  16|    42|    59|  1.22.41.76|https://paytm.com...|             0|        1|
|1437586901|  17|    41|    41| 1.23.208.26|https://paytm.com...|             0|        1|
|1437588047|  18|     0|    47| 1.23.208.26|http://www.paytm....|    1437586914|        2|
|1437542064|   5|    14|    24| 1.23.36.184|https://paytm.com...|             0|        1|
|1437581498|  16|    11|    38|   1.38.19.8|https://paytm.com...|             0|        1|
|1437555638|   9|     0|    38|  1.38.20.34|https://paytm.com...|             0|        1|

In [226]:
Result_df = sp.sql('select ft.host, ft.url, ft.timestamp, ft.prev_timestamp, ft.hour, rt.Sessionum from First_table ft \
LEFT JOIN Rec_table rt on \
ft.host = rt.host and ft.timestamp = rt.timestamp and ft.url = rt.url order by ft.host, ft.timestamp')

Result_df.createOrReplaceTempView('Result')

In [227]:
Result_df.show()


+------------+--------------------+----------+--------------+----+---------+
|        host|                 url| timestamp|prev_timestamp|hour|Sessionum|
+------------+--------------------+----------+--------------+----+---------+
|1.186.101.79|https://paytm.com...|1437561955|             0|  10|        1|
|1.186.101.79|https://paytm.com...|1437561955|    1437561955|  10|     null|
|1.186.101.79|https://paytm.com...|1437561987|    1437561955|  10|     null|
|1.186.101.79|https://paytm.com...|1437562016|    1437561987|  10|     null|
|1.186.101.79|https://paytm.com...|1437562021|    1437562016|  10|     null|
|1.186.101.79|https://paytm.com...|1437562026|    1437562021|  10|     null|
|1.186.101.79|https://paytm.com...|1437562027|    1437562027|  10|     null|
|1.186.101.79|https://paytm.com...|1437562027|    1437562026|  10|     null|
|1.186.101.79|https://paytm.com...|1437562038|    1437562027|  10|     null|
|1.186.101.79|https://paytm.com...|1437562048|    1437562038|  10|     null|

<b>Sessionize the web log by IP. Sessionize = aggregrate all page hits by visitor/IP during a session</b>

In [228]:

import sys
from pyspark.sql.window import Window
import pyspark.sql.functions as func
Sessions_df = Result_df.withColumn("Sessionum_2", \
                                   func.last('Sessionum', True).over(Window.partitionBy('host').orderBy('timestamp').rowsBetween(-sys.maxsize, 0)))


In [229]:
Sessions_df.show(50)

+------------+--------------------+----------+--------------+----+---------+-----------+
|        host|                 url| timestamp|prev_timestamp|hour|Sessionum|Sessionum_2|
+------------+--------------------+----------+--------------+----+---------+-----------+
|1.186.143.37|https://paytm.com...|1437581676|             0|  16|        1|          1|
|1.186.143.37|https://paytm.com...|1437581684|    1437581676|  16|     null|          1|
|1.187.164.29|https://paytm.com...|1437533011|             0|   2|        1|          1|
|1.187.164.29|https://paytm.com...|1437533022|    1437533011|   2|     null|          1|
|1.187.164.29|https://paytm.com...|1437533027|    1437533022|   2|     null|          1|
|1.187.164.29|https://paytm.com...|1437533059|    1437533027|   2|     null|          1|
|1.187.164.29|https://paytm.com...|1437533060|    1437533059|   2|     null|          1|
|1.187.164.29|https://paytm.com...|1437533061|    1437533060|   2|     null|          1|
|1.187.164.29|https:/

In [230]:
newDf = Sessions_df.withColumn("new_prev", when(Sessions_df["prev_timestamp"] == 0,Sessions_df['timestamp']).\
                               when(Sessions_df["prev_timestamp"] != 0, Sessions_df['prev_timestamp']).otherwise(Sessions_df['prev_timestamp'] )) 
                                                                                                
#newDf.show()                                                                                          

In [231]:
newDf = newDf.withColumn('gap', newDf['timestamp'] - newDf['new_prev'])


<b>Determine total URL visits per session</b>

In [232]:
#No. of total Url hits in each session by each host 
Total_url = newDf.groupBy('host', 'Sessionum_2').agg(count('url'))

Total_url.show()

+------------+-----------+----------+
|        host|Sessionum_2|count(url)|
+------------+-----------+----------+
|1.186.143.37|          1|         2|
|1.187.164.29|          1|         9|
|  1.22.41.76|          1|         8|
| 1.23.208.26|          1|         5|
| 1.23.208.26|          2|         1|
| 1.23.36.184|          1|         4|
|   1.38.19.8|          1|         2|
|  1.38.20.34|          1|        35|
|  1.39.13.13|          1|         2|
| 1.39.32.249|          1|         4|
| 1.39.32.249|          2|         2|
|  1.39.32.59|          1|         1|
| 1.39.33.153|          1|         8|
|  1.39.33.33|          1|         2|
|  1.39.33.77|          1|         2|
|  1.39.33.77|          2|         4|
|   1.39.34.4|          1|         1|
|  1.39.40.43|          1|         6|
|  1.39.60.37|          1|        30|
|  1.39.60.37|          2|        19|
+------------+-----------+----------+
only showing top 20 rows



<b>Determine unique URL visits per session. To clarify, count a hit to a unique URL only once per session</b>

In [66]:
unique_url = newDf.groupBy('host', 'Sessionum_2').agg(countDistinct('url'))


+------------+-----------+-------------------+
|        host|Sessionum_2|count(DISTINCT url)|
+------------+-----------+-------------------+
|1.186.143.37|          1|                  2|
|1.187.164.29|          1|                  8|
|  1.22.41.76|          1|                  5|
| 1.23.208.26|          1|                  5|
| 1.23.208.26|          2|                  1|
| 1.23.36.184|          1|                  4|
|   1.38.19.8|          1|                  1|
|  1.38.20.34|          1|                 14|
|  1.39.13.13|          1|                  2|
| 1.39.32.249|          1|                  4|
| 1.39.32.249|          2|                  2|
|  1.39.32.59|          1|                  1|
| 1.39.33.153|          1|                  6|
|  1.39.33.33|          1|                  2|
|  1.39.33.77|          1|                  2|
|  1.39.33.77|          2|                  4|
|   1.39.34.4|          1|                  1|
|  1.39.40.43|          1|                  2|
|  1.39.60.37

<b>Determine the average session time</b>

In [118]:
session_time = newDf.groupBy('host', 'Sessionum_2').agg(sum('gap'))

session_time.show()
#newDf.printSchema()
#Sessions_df.show()

+------------+-----------+--------+
|        host|Sessionum_2|sum(gap)|
+------------+-----------+--------+
|1.186.143.37|          1|       8|
|1.187.164.29|          1|      69|
|  1.22.41.76|          1|      62|
| 1.23.208.26|          1|      13|
| 1.23.208.26|          2|    1133|
| 1.23.36.184|          1|      15|
|   1.38.19.8|          1|      60|
|  1.38.20.34|          1|     192|
|  1.39.13.13|          1|       5|
| 1.39.32.249|          1|      57|
| 1.39.32.249|          2|   20507|
|  1.39.32.59|          1|       0|
| 1.39.33.153|          1|      41|
|  1.39.33.33|          1|       7|
|  1.39.33.77|          1|       2|
|  1.39.33.77|          2|    4713|
|   1.39.34.4|          1|       0|
|  1.39.40.43|          1|     170|
|  1.39.60.37|          1|     254|
|  1.39.60.37|          2|     849|
+------------+-----------+--------+
only showing top 20 rows



<b>Overall average session time</b>

In [139]:
session_time.agg({'sum(gap)' : 'mean'}).show()

+------------------+
|     avg(sum(gap))|
+------------------+
|1979.4600535136822|
+------------------+



<b>mean session time per user</b>

In [147]:
avg_session_time = session_time.groupBy('host').mean('sum(gap)')
avg_session_time.columns
#avg_session_time.show()

['host', 'avg(sum(gap))']

<b>Find the most engaged users, ie the IPs with the longest session times</b>

In [152]:
avg_session_time.sort(desc('avg(sum(gap))')).show()

+---------------+-------------+
|           host|avg(sum(gap))|
+---------------+-------------+
|   27.120.106.3|      33149.5|
| 98.230.153.173|      28711.5|
|117.255.253.155|      28711.5|
|   14.139.69.64|      28693.0|
|  103.24.125.26|      28683.5|
| 150.228.40.140|      28611.0|
|  66.249.82.186|      28564.0|
| 37.228.107.126|      27751.5|
|  163.47.14.170|      27742.5|
| 168.235.194.47|      27739.5|
|    1.39.12.226|      27728.5|
|     1.39.35.89|      27711.5|
|    1.39.63.157|      27705.5|
| 192.20.246.138|      27703.0|
|    59.177.1.75|      27689.0|
|  163.47.12.254|      27681.0|
|    1.39.60.122|      27668.0|
|     1.39.32.67|      27662.0|
|101.221.134.218|      27660.0|
| 107.167.109.55|      27659.0|
+---------------+-------------+
only showing top 20 rows



In [154]:
avg_session_time.sort(desc('avg(sum(gap))').alias('Average session time'))

DataFrame[host: string, avg(sum(gap)): double]

<b>Predict the expected load (requests/second) in the next minute</b>

In [155]:
#Lets make a table for load at each timestamp 
newDf.show()

+------------+--------------------+----------+--------------+---------+-----------+----------+---+
|        host|                 url| timestamp|prev_timestamp|Sessionum|Sessionum_2|  new_prev|gap|
+------------+--------------------+----------+--------------+---------+-----------+----------+---+
|1.186.143.37|https://paytm.com...|1437581676|             0|        1|          1|1437581676|  0|
|1.186.143.37|https://paytm.com...|1437581684|    1437581676|     null|          1|1437581676|  8|
|1.187.164.29|https://paytm.com...|1437533011|             0|        1|          1|1437533011|  0|
|1.187.164.29|https://paytm.com...|1437533022|    1437533011|     null|          1|1437533011| 11|
|1.187.164.29|https://paytm.com...|1437533027|    1437533022|     null|          1|1437533022|  5|
|1.187.164.29|https://paytm.com...|1437533059|    1437533027|     null|          1|1437533027| 32|
|1.187.164.29|https://paytm.com...|1437533060|    1437533059|     null|          1|1437533059|  1|
|1.187.164

In [170]:
#min and max timestamp
newDf.select([min('timestamp').alias('start_time'), max('timestamp').alias('end_time')]).show()

+----------+----------+
|start_time|  end_time|
+----------+----------+
|1437532806|1437599427|
+----------+----------+



In [214]:
#total data spread in 18.5 hours
(1437599427-1437532806)/60

1110.35

In [238]:
#This is the load (request/second) table 
load_per_timestamp = newDf.groupBy('timestamp').agg(count('timestamp').alias('Load')).orderBy('timestamp')

load_per_timestamp.show()

+----------+----+
| timestamp|Load|
+----------+----+
|1437532806|  27|
|1437532807|  62|
|1437532808|  56|
|1437532809| 112|
|1437532810|  58|
|1437532811|  58|
|1437532812|  67|
|1437532813|  85|
|1437532814| 160|
|1437532815|  57|
|1437532816|  99|
|1437532817|  96|
|1437532818|  95|
|1437532819|  65|
|1437532820| 130|
|1437532821|  65|
|1437532822|  59|
|1437532823|  62|
|1437532824|  89|
|1437532825| 130|
+----------+----+
only showing top 20 rows



In [181]:
load_per_timestamp.count()

4269

In [184]:
load_per_timestamp.summary().show()

+-------+--------------------+------------------+
|summary|           timestamp|             count|
+-------+--------------------+------------------+
|  count|                4269|              4269|
|   mean|1.4375682853865073E9|  271.375029280862|
| stddev|  18883.621023404285|142.97318597127335|
|    min|          1437532806|                 1|
|    25%|          1437555781|               175|
|    50%|          1437562977|               304|
|    75%|          1437583395|               374|
|    max|          1437599427|               884|
+-------+--------------------+------------------+



In [189]:
load_per_timestamp = load_per_timestamp.withColumn('new_timestamp', (load_per_timestamp['timestamp'] - 1437532806))
load_per_timestamp.summary().show()

+-------+--------------------+------------------+------------------+
|summary|           timestamp|             count|     new_timestamp|
+-------+--------------------+------------------+------------------+
|  count|                4269|              4269|              4269|
|   mean|1.4375682853865073E9|  271.375029280862|35479.386507378775|
| stddev|  18883.621023404285|142.97318597127335| 18883.62102329229|
|    min|          1437532806|                 1|                 0|
|    25%|          1437555781|               175|             22975|
|    50%|          1437562977|               304|             30171|
|    75%|          1437583395|               374|             50589|
|    max|          1437599427|               884|             66621|
+-------+--------------------+------------------+------------------+



In [267]:
load_per_timestamp.createOrReplaceTempView('loadtable')


In [200]:
#To see that data is not equally spaced in time
sp.sql('select distinct(diffs) from (select new_timestamp, \
(new_timestamp - (lag(new_timestamp,1,0) over (partition by timestamp order by timestamp))) \
as diffs  from loadtable ) order by diffs').show()

+-----+
|diffs|
+-----+
|    0|
|    1|
|    2|
|    3|
|    4|
|    5|
|    6|
|    7|
|    8|
|    9|
|   10|
|   11|
|   12|
|   13|
|   14|
|   15|
|   16|
|   17|
|   18|
|   19|
+-----+
only showing top 20 rows



In [ ]:
#Ignore
hist = df.rdd\
  .map(lambda l: l['age'])\
  .histogram([1, 11, 21,31,41,51,61,71,81,91])
This will return a tuple with "age ranges" and their respective observation count, as:

([1, 11, 21, 31, 41, 51, 61, 71, 81, 91],
  [10, 10, 10, 10, 10, 10, 10, 10, 11])
Then you can convert that back to a data frame using:

#Use zip to link age_ranges to their counts
countTuples = zip(hist[0], hist[1])
#make a list from that
ageList = list(map(lambda l: Row(age_range=l[0], count=l[1]), countTuples))
sc.parallelize(ageList).toDF()

In [217]:
#Ignore
hist = load_per_timestamp.rdd.map(lambda l:l['new_timestamp']).histogram(1110)

countTuples = zip(hist[0], hist[1])
timelist = list(map(lambda l: Row(time_range=l[0], count=l[1]), countTuples))
sc.parallelize(timelist).toDF().show()


+-----+------------------+
|count|        time_range|
+-----+------------------+
|   61|               0.0|
|   60| 60.01891891891892|
|   60|120.03783783783784|
|   60|180.05675675675676|
|   60|240.07567567567568|
|    0| 300.0945945945946|
|    0| 360.1135135135135|
|    0|420.13243243243244|
|    0|480.15135135135137|
|    0| 540.1702702702703|
|    0| 600.1891891891892|
|    0| 660.2081081081081|
|    0|  720.227027027027|
|    0|  780.245945945946|
|    0| 840.2648648648649|
|    0| 900.2837837837839|
|    0| 960.3027027027027|
|    0|1020.3216216216216|
|    0|1080.3405405405406|
|    0|1140.3594594594595|
+-----+------------------+
only showing top 20 rows



<b>Predict the session length for a given IP</b>

In [223]:
newDf.groupBy('host', 'Sessionum_2').agg(sum('gap').alias('Session_length')).show()

+------------+-----------+--------------+
|        host|Sessionum_2|Session_length|
+------------+-----------+--------------+
|1.186.143.37|          1|             8|
|1.187.164.29|          1|            69|
|  1.22.41.76|          1|            62|
| 1.23.208.26|          1|            13|
| 1.23.208.26|          2|          1133|
| 1.23.36.184|          1|            15|
|   1.38.19.8|          1|            60|
|  1.38.20.34|          1|           192|
|  1.39.13.13|          1|             5|
| 1.39.32.249|          1|            57|
| 1.39.32.249|          2|         20507|
|  1.39.32.59|          1|             0|
| 1.39.33.153|          1|            41|
|  1.39.33.33|          1|             7|
|  1.39.33.77|          1|             2|
|  1.39.33.77|          2|          4713|
|   1.39.34.4|          1|             0|
|  1.39.40.43|          1|           170|
|  1.39.60.37|          1|           254|
|  1.39.60.37|          2|           849|
+------------+-----------+--------

<b>Predicting session length for an IP as its mean ..It could be mode median as well. It can be cross validated by dividing data
*Another way is to decode the ip address to map to an area and try to model accordingly. Not implementing here</b>

In [224]:
Sl_df = newDf.groupBy('host', 'Sessionum_2').agg(sum('gap').alias('Session_length'))
Sl_df.groupBy('host').agg(avg('Session_length').alias('Average Session length per host')).show()

+------------+-------------------------------+
|        host|Average Session length per host|
+------------+-------------------------------+
|1.186.143.37|                            8.0|
|1.187.164.29|                           69.0|
|  1.22.41.76|                           62.0|
| 1.23.208.26|                          573.0|
| 1.23.36.184|                           15.0|
|   1.38.19.8|                           60.0|
|  1.38.20.34|                          192.0|
|  1.39.13.13|                            5.0|
| 1.39.32.249|                        10282.0|
|  1.39.32.59|                            0.0|
| 1.39.33.153|                           41.0|
|  1.39.33.33|                            7.0|
|  1.39.33.77|                         2357.5|
|   1.39.34.4|                            0.0|
|  1.39.40.43|                          170.0|
|  1.39.60.37|                          551.5|
|  1.39.61.53|                           16.0|
| 1.39.62.227|                           17.0|
| 1.39.63.197

<b>or we can predict hour wise average but it wont make as a session can last for more than an hour </b>

In [233]:
(newDf.groupBy('host', 'hour', 'Sessionum_2').agg(sum('gap').alias('Session_length_vs_hour'))).groupBy('host','hour').\
agg(avg('Session_length_vs_hour').alias('Average Session length per host vs hour'))

DataFrame[host: string, hour: int, Average Session length per host vs hour: double]

<b>Predict the number of unique URL visits by a given IP .. this can come from the mean of some hits or average of hour wise hit </b>

In [234]:
newDf.groupBy('host').agg(countDistinct('url').alias('# of unique url hits')).show()

+---------------+--------------------+
|           host|# of unique url hits|
+---------------+--------------------+
| 113.193.114.25|                  84|
|115.112.250.108|                  85|
|117.203.181.144|                   2|
| 117.205.39.248|                  16|
| 117.207.97.173|                   3|
| 117.241.152.20|                  34|
| 117.247.188.13|                   9|
|   120.61.47.36|                   7|
|123.136.182.137|                  10|
| 124.125.22.218|                  88|
|  14.139.82.134|                 108|
|  202.174.92.10|                  16|
|  202.53.89.132|                 110|
|  27.34.244.251|                 112|
|   27.63.186.72|                   6|
| 59.160.110.163|                  14|
|  61.16.142.162|                  16|
|     1.39.13.13|                   2|
|    103.16.71.9|                  94|
|103.252.165.138|                   1|
+---------------+--------------------+
only showing top 20 rows



<b>number of unique url hits by a given IP by taking average on hours. With this much data I am not sure modelling on hour wise makes sense so probably closest to hourly no. of hit can be the answer </b>

In [236]:
newDf.groupBy('host', 'hour').agg(countDistinct('url').alias('# of unique url hits'))

DataFrame[host: string, hour: int, # of unique url hits: bigint]

<b>Predict the expected load (requests/second) in the next minute</b>

<b>timestamps are not evenly spaced to do any standard timeseries analysis. 
This would either require interpolation of data or using a part of the data.
Other algorithms such as linear regression are not making sense to me as it a time dependent stats</b>

<b>Persistence model/ naive forecast</b>

In [250]:
#Use lag function to predict the next timestamp load
w = Window.orderBy('timestamp')
load_per_timestamp.withColumn('pred_load', lag('Load',1, 0).over(w))

+----------+----+---------+
| timestamp|Load|pred_load|
+----------+----+---------+
|1437532806|  27|        0|
|1437532807|  62|       27|
|1437532808|  56|       62|
|1437532809| 112|       56|
|1437532810|  58|      112|
|1437532811|  58|       58|
|1437532812|  67|       58|
|1437532813|  85|       67|
|1437532814| 160|       85|
|1437532815|  57|      160|
|1437532816|  99|       57|
|1437532817|  96|       99|
|1437532818|  95|       96|
|1437532819|  65|       95|
|1437532820| 130|       65|
|1437532821|  65|      130|
|1437532822|  59|       65|
|1437532823|  62|       59|
|1437532824|  89|       62|
|1437532825| 130|       89|
+----------+----+---------+
only showing top 20 rows



In [276]:
#For next minute total load can be summed up last 60 consecutive timestamps

sp.sql('select sum(Load) from (select * from loadtable L order by timestamp desc LIMIT 60)').show()

+---------+
|sum(Load)|
+---------+
|     5081|
+---------+



In [ ]:
#Other methods such as Moving average smoothing ARMA ARIMA can be used if data is equally spaced to start with. 